In [45]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score


In [46]:
df = pd.read_csv('nbagames.csv', index_col=0)

In [47]:
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,240.0,240.0,35.0,97.0,0.361,8.0,29.0,0.276,17.0,21.0,...,17.0,31.8,147.0,105.0,MIL,109,0,2019,2018-12-25,False
15158,240.0,240.0,34.0,95.0,0.358,7.0,33.0,0.212,17.0,23.0,...,59.2,34.5,231.0,103.0,LAC,108,1,2019,2018-10-19,False
15159,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
15160,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [48]:
print(df['season'].value_counts())

season
2022    2646
2023    2640
2018    2624
2019    2624
2021    2342
2020    2286
Name: count, dtype: int64


In [49]:
df = df.sort_values("date")

In [50]:
df = df.reset_index(drop=True)

In [51]:
del df['mp.1']
del df["mp_opp.1"]
del df["index_opp"]

In [52]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner
0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,...,34.7,29.9,129.0,112.0,BOS,99,0,2018,2017-10-17,True
1,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,...,31.6,27.3,138.0,107.0,CLE,102,1,2018,2017-10-17,False
2,240.0,47.0,97.0,0.485,15.0,41.0,0.366,13.0,19.0,0.684,...,31.7,32.6,250.0,127.0,GSW,121,1,2018,2017-10-17,True
3,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,...,18.2,31.2,152.0,126.0,HOU,122,0,2018,2017-10-17,False
4,240.0,30.0,79.0,0.380,7.0,25.0,0.280,24.0,29.0,0.828,...,40.0,28.8,239.0,111.0,MEM,103,1,2018,2017-10-18,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,240.0,41.0,80.0,0.513,5.0,18.0,0.278,22.0,27.0,0.815,...,11.1,95.2,134.0,129.0,MIA,94,1,2023,2023-06-07,True
15158,240.0,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,0.850,...,50.0,32.6,164.0,114.0,DEN,108,0,2023,2023-06-09,False
15159,240.0,39.0,79.0,0.494,14.0,28.0,0.500,16.0,21.0,0.762,...,25.2,30.3,223.0,127.0,MIA,95,1,2023,2023-06-09,True
15160,240.0,33.0,96.0,0.344,9.0,35.0,0.257,14.0,16.0,0.875,...,28.6,26.4,202.0,104.0,DEN,94,1,2023,2023-06-12,False


In [53]:
def add_target(team):
    team["target"] = team["winner"].shift(-1)
    return team

df = df.groupby("team", group_keys = False).apply(add_target)


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/914483113.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["winner"].shift(-1)
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/914483113.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["winner"].shift(-1)
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/914483113.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

In [54]:
df[df["team"] == "LAL"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner,target
30,240.0,37.0,91.0,0.407,4.0,16.0,0.250,14.0,23.0,0.609,...,46.1,140.0,97.0,LAC,108,0,2018,2017-10-19,False,True
40,240.0,48.0,93.0,0.516,12.0,26.0,0.462,24.0,33.0,0.727,...,28.5,148.0,118.0,PHO,130,1,2018,2017-10-20,True,False
79,240.0,44.0,90.0,0.489,9.0,27.0,0.333,15.0,20.0,0.750,...,32.5,203.0,117.0,NOP,119,0,2018,2017-10-22,False,True
121,265.0,41.0,92.0,0.446,7.0,30.0,0.233,13.0,16.0,0.813,...,30.7,156.0,98.0,WAS,99,0,2018,2017-10-25,True,False
150,240.0,36.0,79.0,0.456,3.0,23.0,0.130,17.0,24.0,0.708,...,33.0,161.0,99.0,TOR,101,0,2018,2017-10-27,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15125,240.0,39.0,75.0,0.520,13.0,26.0,0.500,31.0,42.0,0.738,...,35.8,208.0,135.0,GSW,101,0,2023,2023-05-12,True,False
15130,240.0,46.0,84.0,0.548,11.0,24.0,0.458,23.0,26.0,0.885,...,25.9,157.0,139.0,DEN,132,1,2023,2023-05-16,False,False
15134,240.0,36.0,82.0,0.439,8.0,30.0,0.267,23.0,26.0,0.885,...,31.5,126.0,110.0,DEN,108,1,2023,2023-05-18,False,False
15139,240.0,38.0,84.0,0.452,10.0,32.0,0.313,22.0,29.0,0.759,...,35.2,162.0,118.0,DEN,119,0,2023,2023-05-20,False,False


In [55]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors = "ignore")

/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/497237324.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/4972373

In [56]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner,target
0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,...,29.9,129.0,112.0,BOS,99,0,2018,2017-10-17,True,1
1,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,...,27.3,138.0,107.0,CLE,102,1,2018,2017-10-17,False,0
2,240.0,47.0,97.0,0.485,15.0,41.0,0.366,13.0,19.0,0.684,...,32.6,250.0,127.0,GSW,121,1,2018,2017-10-17,True,1
3,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,...,31.2,152.0,126.0,HOU,122,0,2018,2017-10-17,False,1
4,240.0,30.0,79.0,0.380,7.0,25.0,0.280,24.0,29.0,0.828,...,28.8,239.0,111.0,MEM,103,1,2018,2017-10-18,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,240.0,41.0,80.0,0.513,5.0,18.0,0.278,22.0,27.0,0.815,...,95.2,134.0,129.0,MIA,94,1,2023,2023-06-07,True,1
15158,240.0,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,0.850,...,32.6,164.0,114.0,DEN,108,0,2023,2023-06-09,False,0
15159,240.0,39.0,79.0,0.494,14.0,28.0,0.500,16.0,21.0,0.762,...,30.3,223.0,127.0,MIA,95,1,2023,2023-06-09,True,1
15160,240.0,33.0,96.0,0.344,9.0,35.0,0.257,14.0,16.0,0.875,...,26.4,202.0,104.0,DEN,94,1,2023,2023-06-12,False,2


In [57]:
df["winner"].value_counts()

winner
True     7581
False    7581
Name: count, dtype: int64

In [58]:
df["target"].value_counts()

target
1    7566
0    7566
2      30
Name: count, dtype: int64

In [59]:
nulls = pd.isnull(df).sum()

In [60]:
nulls = nulls[nulls > 0]

In [61]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [62]:
df = df[valid_columns].copy()

In [63]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner,target
0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,0.840,...,29.9,129.0,112.0,BOS,99,0,2018,2017-10-17,True,1
1,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,0.760,...,27.3,138.0,107.0,CLE,102,1,2018,2017-10-17,False,0
2,240.0,47.0,97.0,0.485,15.0,41.0,0.366,13.0,19.0,0.684,...,32.6,250.0,127.0,GSW,121,1,2018,2017-10-17,True,1
3,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,0.905,...,31.2,152.0,126.0,HOU,122,0,2018,2017-10-17,False,1
4,240.0,30.0,79.0,0.380,7.0,25.0,0.280,24.0,29.0,0.828,...,28.8,239.0,111.0,MEM,103,1,2018,2017-10-18,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,240.0,41.0,80.0,0.513,5.0,18.0,0.278,22.0,27.0,0.815,...,95.2,134.0,129.0,MIA,94,1,2023,2023-06-07,True,1
15158,240.0,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,0.850,...,32.6,164.0,114.0,DEN,108,0,2023,2023-06-09,False,0
15159,240.0,39.0,79.0,0.494,14.0,28.0,0.500,16.0,21.0,0.762,...,30.3,223.0,127.0,MIA,95,1,2023,2023-06-09,True,1
15160,240.0,33.0,96.0,0.344,9.0,35.0,0.257,14.0,16.0,0.875,...,26.4,202.0,104.0,DEN,94,1,2023,2023-06-12,False,2


In [64]:
rr = RidgeClassifier(alpha = 1)
split = TimeSeriesSplit(n_splits= 3)

sfs = SequentialFeatureSelector(rr, n_features_to_select= 30, direction = "forward", cv = split)

scaler = MinMaxScaler() #Scales the data to be between 0 and 1 for better interpretation

In [65]:
removed_columns = ["season", "date", "team", "winner", "target", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [66]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,winner,target
0,0.0,0.386364,0.327869,0.452153,0.142857,0.200000,0.217557,0.465116,0.380952,0.813302,...,0.101282,0.149254,0.411765,BOS,0.287037,0.0,2018,2017-10-17,True,1
1,0.0,0.340909,0.409836,0.334928,0.250000,0.366667,0.246819,0.418605,0.380952,0.719953,...,0.067949,0.194030,0.352941,CLE,0.314815,1.0,2018,2017-10-17,False,0
2,0.0,0.590909,0.557377,0.516746,0.500000,0.516667,0.394402,0.279070,0.285714,0.631272,...,0.135897,0.751244,0.588235,GSW,0.490741,1.0,2018,2017-10-17,True,1
3,0.0,0.500000,0.278689,0.643541,0.535714,0.333333,0.606870,0.418605,0.317460,0.889148,...,0.117949,0.263682,0.576471,HOU,0.500000,0.0,2018,2017-10-17,False,1
4,0.0,0.204545,0.262295,0.265550,0.214286,0.250000,0.284987,0.534884,0.444444,0.799300,...,0.087179,0.696517,0.400000,MEM,0.324074,1.0,2018,2017-10-18,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,0.0,0.454545,0.278689,0.583732,0.142857,0.133333,0.282443,0.488372,0.412698,0.784131,...,0.938462,0.174129,0.611765,MIA,0.240741,1.0,2023,2023-06-07,True,1
15158,0.0,0.318182,0.245902,0.430622,0.250000,0.250000,0.335878,0.372093,0.301587,0.824971,...,0.135897,0.323383,0.435294,DEN,0.370370,0.0,2023,2023-06-09,False,0
15159,0.0,0.409091,0.262295,0.538278,0.464286,0.300000,0.564885,0.348837,0.317460,0.722287,...,0.106410,0.616915,0.588235,MIA,0.250000,1.0,2023,2023-06-09,True,1
15160,0.0,0.272727,0.540984,0.179426,0.285714,0.416667,0.255725,0.302326,0.238095,0.854142,...,0.056410,0.512438,0.317647,DEN,0.240741,1.0,2023,2023-06-12,False,2


In [67]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [68]:
predictors = list(selected_columns[sfs.get_support()])
predictors

['mp',
 'ft%',
 'orb',
 'drb',
 'tov',
 'usg%',
 'ft%_max',
 'orb_max',
 'drb_max',
 'blk_max',
 'orb%_max',
 'drb%_max',
 'trb%_max',
 'blk%_max',
 'usg%_max',
 'mp_opp',
 'fg%_opp',
 '3p%_opp',
 'pf_opp',
 'ast%_opp',
 'usg%_opp',
 '3pa_max_opp',
 'ft%_max_opp',
 'pts_max_opp',
 'ts%_max_opp',
 '3par_max_opp',
 'drb%_max_opp',
 'trb%_max_opp',
 'stl%_max_opp',
 'blk%_max_opp']

In [69]:
def backtest(data,model,predictors,start = 2, step = 1):

    all_preds = []
    seasons = sorted(data["season"].unique())

    for i in range(start, len(seasons), step):

        season = seasons[i]

        train = data[data["season"] < season]
        test = data[data["season"] == season]

        model.fit(train[predictors], train["target"])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index = test.index)

        combined = pd.concat([test["target"], preds], axis = 1)
        combined.columns = ["actual", "prediction"]
        all_preds.append(combined)
    
    return pd.concat(all_preds)


In [70]:
predictions = backtest(df,rr,predictors)

In [71]:
predictions

,actual,prediction
5248,1,0
5249,1,1
5250,0,0
5251,0,0
5252,1,0
...,...,...
15157,1,1
15158,0,0
15159,1,1
15160,2,0


In [72]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.5399435142223119

In [73]:
df.groupby("home").apply(lambda x: x[x["winner"] == 1].shape[0] / x.shape[0])

/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/2417177499.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("home").apply(lambda x: x[x["winner"] == 1].shape[0] / x.shape[0])


home
0.0    0.432529
1.0    0.567471
dtype: float64

In [74]:
df_rolling = df[list(selected_columns) + ["winner", "team", "season"]]

def find_team_averages(team):
    rolling = team[selected_columns].rolling(10).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False)
df_rolling = df_rolling.apply(find_team_averages)


/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/3665767723.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.apply(find_team_averages)


In [75]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,0.0,0.500000,0.349180,0.576794,0.357143,0.313333,0.405344,0.418605,0.352381,0.785998,...,0.285636,0.322887,0.0461,0.0710,0.314361,0.340897,0.382587,0.642353,0.367593,0.5
15158,0.0,0.393182,0.363934,0.439952,0.400000,0.346667,0.431298,0.316279,0.260317,0.829988,...,0.162390,0.383568,0.0452,0.0910,0.262998,0.246154,0.465174,0.535294,0.342593,0.5
15159,0.0,0.475000,0.332787,0.560526,0.382143,0.323333,0.428499,0.416279,0.349206,0.788681,...,0.282785,0.305399,0.0458,0.0883,0.319811,0.340769,0.396517,0.628235,0.336111,0.5
15160,0.0,0.372727,0.370492,0.412919,0.400000,0.361667,0.419975,0.306977,0.255556,0.821587,...,0.162390,0.409272,0.0465,0.1108,0.257547,0.237949,0.488060,0.512941,0.332407,0.5


In [76]:
rolling_cols = [f"{col}_10avg" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis = 1)


In [77]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,trb%_max_opp_10avg,ast%_max_opp_10avg,stl%_max_opp_10avg,blk%_max_opp_10avg,tov%_max_opp_10avg,usg%_max_opp_10avg,ortg_max_opp_10avg,drtg_max_opp_10avg,total_opp_10avg,home_opp_10avg
0,0.0,0.386364,0.327869,0.452153,0.142857,0.200000,0.217557,0.465116,0.380952,0.813302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.340909,0.409836,0.334928,0.250000,0.366667,0.246819,0.418605,0.380952,0.719953,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.590909,0.557377,0.516746,0.500000,0.516667,0.394402,0.279070,0.285714,0.631272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.500000,0.278689,0.643541,0.535714,0.333333,0.606870,0.418605,0.317460,0.889148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.204545,0.262295,0.265550,0.214286,0.250000,0.284987,0.534884,0.444444,0.799300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15157,0.0,0.454545,0.278689,0.583732,0.142857,0.133333,0.282443,0.488372,0.412698,0.784131,...,0.285636,0.322887,0.0461,0.0710,0.314361,0.340897,0.382587,0.642353,0.367593,0.5
15158,0.0,0.318182,0.245902,0.430622,0.250000,0.250000,0.335878,0.372093,0.301587,0.824971,...,0.162390,0.383568,0.0452,0.0910,0.262998,0.246154,0.465174,0.535294,0.342593,0.5
15159,0.0,0.409091,0.262295,0.538278,0.464286,0.300000,0.564885,0.348837,0.317460,0.722287,...,0.282785,0.305399,0.0458,0.0883,0.319811,0.340769,0.396517,0.628235,0.336111,0.5
15160,0.0,0.272727,0.540984,0.179426,0.285714,0.416667,0.255725,0.302326,0.238095,0.854142,...,0.162390,0.409272,0.0465,0.1108,0.257547,0.237949,0.488060,0.512941,0.332407,0.5


In [78]:
df = df.dropna()

In [79]:
df = df.copy()

In [80]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):

    return df.groupby("team", group_keys= False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/3995805504.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys= False).apply(lambda x: shift_col(x, col_name))
/var/folders/rt/vlxzyc6d2mggxf0fjtgxlls80000gn/T/ipykernel_47875/3995805504.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys= False).apply(lambda x: shift_co

In [81]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]],
                left_on = ["team", "date_next"], 
                right_on = ["team_opp_next", "date_next"])

In [82]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,DEN,BRK,BRK,DEN,2017-11-07
1,DET,IND,IND,DET,2017-11-08
2,NOP,IND,IND,NOP,2017-11-07
3,MEM,POR,POR,MEM,2017-11-07
4,MIN,GSW,GSW,MIN,2017-11-08
...,...,...,...,...,...
13443,DEN,MIA,MIA,DEN,2023-06-07
13444,MIA,DEN,DEN,MIA,2023-06-09
13445,DEN,MIA,MIA,DEN,2023-06-09
13446,MIA,DEN,DEN,MIA,2023-06-12


In [85]:
full.columns[full.dtypes == "object"]

Index(['team_x', 'team_opp', 'date', 'team_opp_next_x', 'date_next',
       'team_opp_next_y', 'team_y'],
      dtype='object')

In [86]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns


In [87]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_opp_next_x',
 'date_next',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'team',
 'winner',
 'target',
 'team_opp']

In [88]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]


In [89]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [90]:
predictors = list(selected_columns[sfs.get_support()])

In [91]:
predictors

['usg%',
 'ft%_max',
 'blk_max',
 'stl%_max',
 '3pa_opp',
 '3p%_opp',
 'usg%_opp',
 'efg%_max_opp',
 'usg%_max_opp',
 '3pa_10avg_x',
 '3par_10avg_x',
 'trb%_10avg_x',
 'usg%_10avg_x',
 'ortg_10avg_x',
 'drb_max_10avg_x',
 'gmsc_max_10avg_x',
 'blk_opp_10avg_x',
 'trb%_opp_10avg_x',
 'usg%_opp_10avg_x',
 '3pa_max_opp_10avg_x',
 'pf_max_opp_10avg_x',
 'ts%_max_opp_10avg_x',
 'home_next',
 'fg%_10avg_y',
 'usg%_10avg_y',
 'ortg_10avg_y',
 'ast_max_10avg_y',
 'gmsc_max_10avg_y',
 'blk_opp_10avg_y',
 'usg%_opp_10avg_y']

In [92]:
predictions = backtest(full,rr,predictors)


In [93]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.6235173357664233